# Getting Started Guide

## Using Coach from the Command Line

When running Coach from the command line, we use a Preset module to define the experiment parameters.
As its name implies, a preset is a predefined set of parameters to run some agent on some environment.
Coach has many predefined presets that follow the algorithms definitions in the published papers, and allows training some of the existing algorithms with essentially no coding at all. This presets can easily be run from the command line. For example:

`coach -p CartPole_DQN`

You can find all the predefined presets under the `presets` directory, or by listing them using the following command:

`coach -l`

Coach can also be used with an externally defined preset by passing the absolute path to the module and the name of the graph manager object which is defined in the preset: 

`coach -p /home/my_user/my_agent_dir/my_preset.py:graph_manager`

Some presets are generic for multiple environment levels, and therefore require defining the specific level through the command line:

`coach -p Atari_DQN -lvl breakout`

There are plenty of other command line arguments you can use in order to customize the experiment. A full documentation of the available arguments can be found using the following command:

`coach -h`

## Using Coach as a Library

Alternatively, Coach can be used a library directly from python. As described above, Coach uses the presets mechanism to define the experiments. A preset is essentially a python module which instantiates a `GraphManager` object. The graph manager is a container that holds the agents and the environments, and has some additional parameters for running the experiment, such as visualization parameters. The graph manager acts as the scheduler which orchestrates the experiment.

Running Coach directly from python is done through a `CoachInterface` object, which uses the same arguments as the command line invocation but allowes for more flexibility and additional control of the training/inference process.

Let's start with some examples.

Creating a very simple graph containing a single Clipped PPO agent running with the CartPole-v0 Gym environment:

In [ ]:
# Adding module path to sys path if not there, so rl_coach submodules can be imported
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
resources_path = os.path.abspath(os.path.join('Resources'))
if module_path not in sys.path:
    sys.path.append(module_path)
if resources_path not in sys.path:
    sys.path.append(resources_path)

In [ ]:
from rl_coach.coach import CoachInterface

coach = CoachInterface(preset='CartPole_ClippedPPO',
                       custom_parameter='heatup_steps=EnvironmentSteps(5);improve_steps=TrainingSteps(3)')

Running the graph according to the given schedule:

In [ ]:
coach.run()

### Running each phase manually

The graph manager (which was instantiated in the preset) can be accessed from the `CoachInterface` object. The graph manager simplifies the scheduling process by encapsulating the calls to each of the training phases. Sometimes, it can be beneficial to have a more fine grained control over the scheduling process. This can be easily done by calling the individual phase functions directly:

In [ ]:
from rl_coach.core_types import EnvironmentSteps

coach.graph_manager.heatup(EnvironmentSteps(100))
for _ in range(10):
    coach.graph_manager.train_and_act(EnvironmentSteps(50))

### Additional functionality

`CoachInterface` allows for easy access to functionalities such as multi-threading and saving checkpoints:

In [ ]:
coach = CoachInterface(preset='CartPole_ClippedPPO', num_workers=2, checkpoint_save_secs=10)
coach.run()

### Agent functionality

When using `CoachInterface` (single agent with one level of hierarchy) it's also possible to easily use the `Agent` object functionality, such as logging and reading signals and applying the policy the agent has learned on a given state:

In [ ]:
from rl_coach.environments.gym_environment import GymEnvironment,  GymVectorEnvironment
from rl_coach.base_parameters import VisualizationParameters
from rl_coach.core_types import EnvironmentSteps

coach = CoachInterface(preset='CartPole_ClippedPPO')

# training
for it in range(10):
    coach.graph_manager.log_signal('iteration', it)
    coach.graph_manager.train_and_act(EnvironmentSteps(100))
    training_reward = coach.graph_manager.get_signal_value('Training Reward')

In [ ]:
# inference
env_params = GymVectorEnvironment(level='CartPole-v0')
env = GymEnvironment(**env_params.__dict__, visualization_parameters=VisualizationParameters())

for it in range(10):
    action_info = coach.graph_manager.get_agent().choose_action(env.state)
    print("State:{}, Action:{}".format(env.state,action_info.action))
    env.step(action_info.action)

## Using GraphManager Directly

It is also possible to invoke coach directly in the python code without defining a preset (which is necessary for `CoachInterface`) by using the `GraphManager` object directly. Using Coach this way won't allow you access functionalities such as multi-threading, but it might be convenient if you don't want to define a preset file.

Here we show an example of how to do so with a custom environment.
We can use a custom gym environment without registering it. 
We just need the path to the environment module.
We can also pass custom parameters for the environment `__init__` function as `additional_simulator_parameters`.

In [ ]:
from rl_coach.agents.clipped_ppo_agent import ClippedPPOAgentParameters
from rl_coach.environments.gym_environment import GymVectorEnvironment
from rl_coach.graph_managers.basic_rl_graph_manager import BasicRLGraphManager
from rl_coach.graph_managers.graph_manager import SimpleSchedule
from rl_coach.architectures.embedder_parameters import InputEmbedderParameters

# define the environment parameters
bit_length = 10
env_params = GymVectorEnvironment(level='rl_coach.environments.toy_problems.bit_flip:BitFlip')
env_params.additional_simulator_parameters = {'bit_length': bit_length, 'mean_zero': True}

# Clipped PPO
agent_params = ClippedPPOAgentParameters()
agent_params.network_wrappers['main'].input_embedders_parameters = {
    'state': InputEmbedderParameters(scheme=[]),
    'desired_goal': InputEmbedderParameters(scheme=[])
}

graph_manager = BasicRLGraphManager(
    agent_params=agent_params,
    env_params=env_params,
    schedule_params=SimpleSchedule()
)

In [ ]:
graph_manager.improve()

The path to the environment can also be set as an absolute path, as follows: `<absolute python module path>:<environment class>`. For example:

In [ ]:
env_params = GymVectorEnvironment(level='/home/user/my_environment_dir/my_environment_module.py:MyEnvironmentClass')

### Advanced functionality - proprietary exploration policy, checkpoint evaluation

Agent modules, such as exploration policy, memory and neural network topology can be replaced with proprietary ones. In this example we'll show how to replace the default exploration policy of the DQN agent with a different one that is defined under the Resources folder. We'll also show how to change the default checkpoint save settings, and how to load a checkpoint for evaluation.

We'll start with the standard definitions of a DQN agent solving the CartPole environment (taken from the Cartpole_DQN preset)

In [ ]:
from rl_coach.agents.dqn_agent import DQNAgentParameters
from rl_coach.base_parameters import VisualizationParameters, TaskParameters
from rl_coach.core_types import TrainingSteps, EnvironmentEpisodes, EnvironmentSteps
from rl_coach.environments.gym_environment import GymVectorEnvironment
from rl_coach.graph_managers.basic_rl_graph_manager import BasicRLGraphManager
from rl_coach.graph_managers.graph_manager import ScheduleParameters
from rl_coach.memories.memory import MemoryGranularity


####################
# Graph Scheduling #
####################

schedule_params = ScheduleParameters()
schedule_params.improve_steps = TrainingSteps(4000)
schedule_params.steps_between_evaluation_periods = EnvironmentEpisodes(10)
schedule_params.evaluation_steps = EnvironmentEpisodes(1)
schedule_params.heatup_steps = EnvironmentSteps(1000)

#########
# Agent #
#########
agent_params = DQNAgentParameters()

# DQN params
agent_params.algorithm.num_steps_between_copying_online_weights_to_target = EnvironmentSteps(100)
agent_params.algorithm.discount = 0.99
agent_params.algorithm.num_consecutive_playing_steps = EnvironmentSteps(1)

# NN configuration
agent_params.network_wrappers['main'].learning_rate = 0.00025
agent_params.network_wrappers['main'].replace_mse_with_huber_loss = False

# ER size
agent_params.memory.max_size = (MemoryGranularity.Transitions, 40000)

################
#  Environment #
################
env_params = GymVectorEnvironment(level='CartPole-v0')

Next, we'll override the exploration policy with our own policy defined in `Resources/exploration.py`.
We'll also define the checkpoint save directory and interval in seconds.

Make sure the first cell at the top of this notebook is run before the following one, such that module_path and resources_path are adding to sys path.

In [ ]:
from exploration import MyExplorationParameters

# Overriding the default DQN Agent exploration policy with my exploration policy
agent_params.exploration = MyExplorationParameters()

# Creating a graph manager to train a DQN agent to solve CartPole
graph_manager = BasicRLGraphManager(agent_params=agent_params, env_params=env_params,
                                    schedule_params=schedule_params, vis_params=VisualizationParameters())

# Resources path was defined at the top of this notebook
my_checkpoint_dir = resources_path + '/checkpoints'

# Checkpoints will be stored every 5 seconds to the given directory
task_parameters1 = TaskParameters()
task_parameters1.checkpoint_save_dir = my_checkpoint_dir
task_parameters1.checkpoint_save_secs = 5

graph_manager.create_graph(task_parameters1)
graph_manager.improve()


Last, we'll load the latest checkpoint from the checkpoint directory, and evaluate it.

In [ ]:
import tensorflow as tf
import shutil

# Clearing the previous graph before creating the new one to avoid name conflicts
tf.reset_default_graph()

# Updating the graph manager's task parameters to restore the latest stored checkpoint from the checkpoints directory
task_parameters2 = TaskParameters()
task_parameters2.checkpoint_restore_path = my_checkpoint_dir

graph_manager.create_graph(task_parameters2)
graph_manager.evaluate(EnvironmentSteps(5))

# Clearning up
shutil.rmtree(my_checkpoint_dir)